# PAISE 2025 Plots/Analysis

## Preamble

In [ ]:
# Common imports.
from pathlib import Path
from collections import defaultdict

%matplotlib widget
import base

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

import dfaas_env
import dfaas_utils
import dfaas_upperbound

In [ ]:
# Increase font, fig sizes and DPI for plots.
plt.rcParams.update(
    {
        "figure.figsize": [7.68, 5.76],  # Increased by 20% from the default [6.4, 4.8]
        "figure.dpi": 150,  # Increase DPI value from the default (140)
        "font.size": 15,  # Increase the default font size (12/11)
    }
)

# Training

## Experiment loading

In [ ]:
prefix_dir = Path("../results/paise2025_2")

exps_dir = []
for exp in prefix_dir.iterdir():
    exps_dir.append(exp)

# DEBUG
# exps_dir = [prefix_dir / "DFAAS-MA_2025-02-13_02-11-18_PPO_45"]

assert len(exps_dir) > 0, "must select at least one experiment"

# Preload the data (result.json file) for all selected experiments.
raw_exp_data = {}
for exp_dir in exps_dir:
    raw_exp_data[exp_dir] = dfaas_utils.parse_result_file(exp_dir / "result.json")

# Create the reference environment based on DFaaS.
for exp_dir in exps_dir:
    env = base.get_env(exp_dir)
    if env.__class__ == dfaas_env.DFaaS:
        break

# At least one experiment must be of type DFaaS. SingleDFaaS is used only as
# reference.
assert env.__class__ == dfaas_env.DFaaS, f"{env.__class__}"

print("Selected experiments:")
for exp_dir in exps_dir:
    print(f"  - {exp_dir.name}")

## Enqueued requests

In [ ]:
# Common functions for processed requests.


def get_enqueued_requests_data_upperbound(iters):
    iterations = len(iters)
    final_data = {}
    final_data["input_reqs_avg"] = np.empty(iterations)
    final_data["enqueued_reqs_local_avg"] = np.empty(iterations)
    final_data["enqueued_reqs_forward_avg"] = np.empty(iterations)

    # For each iteration, calculate the metrics for each episode played,
    # then average the values for the number of episodes of that iteration.
    for iter_idx in range(iterations):
        episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]
        iter_data = iters[iter_idx]["env_runners"]["hist_stats"]

        # Create the data dictionary that contains the metrics for each
        # episode in this iteration.
        data = {}
        data["input_reqs"] = np.empty(episodes, dtype=np.int32)
        data["enqueued_reqs_local"] = np.empty(episodes, dtype=np.int32)
        data["enqueued_reqs_forward"] = np.empty(episodes, dtype=np.int32)

        # Iterate the episodes.
        for epi_idx in range(episodes):
            input_reqs = np.sum(iter_data["observation_input_requests"][epi_idx])

            # Calculate local, queued, or directly processed requests. Note that the
            # rejections must be subtracted!
            local_reqs = np.array(iter_data["action_local"][epi_idx])
            local_rejects = np.array(iter_data["local_rejects_queue_full"][epi_idx])
            real_local_reqs = np.sum(local_reqs - local_rejects)

            data["input_reqs"][epi_idx] = input_reqs
            data["enqueued_reqs_local"][epi_idx] = real_local_reqs

        # Update iteration data.
        final_data["input_reqs_avg"][iter_idx] = np.average(data["input_reqs"])
        final_data["enqueued_reqs_local_avg"][iter_idx] = np.average(
            data["enqueued_reqs_local"]
        )

    return final_data


def get_enqueued_requests_data(iter_data):
    # (experiment, iteration, metrics).
    final_data = defaultdict(lambda: defaultdict(lambda: defaultdict()))

    for exp_dir, iters in iter_data.items():
        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The GlobalNode data extraction is different from the normal flow.
            final_data[exp_dir] = get_enqueued_requests_data_upperbound(iters)
            continue

        iterations = len(iters)

        # Create the portion of the dictionary for this experiment that
        # contains the average values of the metrics for each iteration.
        for agent in ["all"] + env.agents:
            final_data[exp_dir][agent]["input_reqs_avg"] = np.empty(iterations)
            final_data[exp_dir][agent]["enqueued_reqs_local_avg"] = np.empty(iterations)
            final_data[exp_dir][agent]["enqueued_reqs_forward_avg"] = np.empty(
                iterations
            )
            final_data[exp_dir][agent]["enqueued_reqs_reject_avg"] = np.empty(
                iterations
            )

        # For each iteration, calculate the metrics for each episode played,
        # then average the values for the number of episodes of that iteration.
        for iter_idx in range(iterations):
            episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]
            iter_data = iters[iter_idx]["env_runners"]["hist_stats"]

            # Create the data dictionary that contains the metrics for each
            # episode in this iteration.
            data = defaultdict(lambda: defaultdict())
            for agent in ["all"] + env.agents:
                data[agent]["input_reqs"] = np.zeros(episodes, dtype=np.int32)
                data[agent]["enqueued_reqs_local"] = np.zeros(episodes, dtype=np.int32)
                data[agent]["enqueued_reqs_forward"] = np.zeros(
                    episodes, dtype=np.int32
                )
                data[agent]["enqueued_reqs_reject"] = np.zeros(episodes, dtype=np.int32)

            # Iterate the episodes.
            for epi_idx in range(episodes):
                for agent in env.agents:
                    input_reqs = np.sum(
                        iter_data["observation_input_requests"][epi_idx][agent]
                    )

                    # Calculate local, queued, or directly processed requests. Note that the
                    # rejections must be subtracted!
                    local_reqs = np.array(iter_data["action_local"][epi_idx][agent])
                    local_rejects = np.array(
                        iter_data["local_rejects_queue_full"][epi_idx][agent]
                    )
                    real_local_reqs = np.sum(local_reqs - local_rejects)

                    # Same for forwarded requests.
                    forwarded_reqs = np.array(
                        iter_data["action_forward"][epi_idx][agent]
                    )
                    forwarded_rejects = np.array(
                        iter_data["forward_rejects"][epi_idx][agent]
                    )
                    real_forwarded_reqs = np.sum(forwarded_reqs - forwarded_rejects)

                    rejected_reqs = np.array(iter_data["action_reject"][epi_idx][agent])
                    real_rejected_reqs = np.sum(
                        rejected_reqs + local_rejects + forwarded_rejects
                    )

                    assert np.all(
                        real_local_reqs + real_forwarded_reqs + real_rejected_reqs
                        == input_reqs
                    )

                    data[agent]["input_reqs"][epi_idx] = input_reqs
                    data[agent]["enqueued_reqs_local"][epi_idx] = real_local_reqs
                    data[agent]["enqueued_reqs_forward"][epi_idx] = real_forwarded_reqs
                    data[agent]["enqueued_reqs_reject"][epi_idx] = real_rejected_reqs
                    data["all"]["input_reqs"][epi_idx] += input_reqs
                    data["all"]["enqueued_reqs_local"][epi_idx] += real_local_reqs
                    data["all"]["enqueued_reqs_forward"][epi_idx] += real_forwarded_reqs
                    data["all"]["enqueued_reqs_reject"][epi_idx] += real_rejected_reqs

            # Update iteration data.
            for agent in ["all"] + env.agents:
                final_data[exp_dir][agent]["input_reqs_avg"][iter_idx] = np.average(
                    data[agent]["input_reqs"]
                )
                final_data[exp_dir][agent]["enqueued_reqs_local_avg"][iter_idx] = (
                    np.average(data[agent]["enqueued_reqs_local"])
                )
                final_data[exp_dir][agent]["enqueued_reqs_forward_avg"][iter_idx] = (
                    np.average(data[agent]["enqueued_reqs_forward"])
                )
                final_data[exp_dir][agent]["enqueued_reqs_reject_avg"][iter_idx] = (
                    np.average(data[agent]["enqueued_reqs_reject"])
                )

    return final_data


enqueued_reqs_data = get_enqueued_requests_data(raw_exp_data)

In [ ]:
# Postprocess additional data.
enqueued_reqs_data_avg = {"SAC": {}, "PPO": {}, "APL": {}, "GlobalNode": {}}
algorithms = ["SAC", "PPO", "APL"]
for algorithm in algorithms:
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms (and environment).
        if algorithm in exp_dir.name and "SingleDFaaS" not in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        continue

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            enqueued_reqs_data[exp]["all"]["enqueued_reqs_local_avg"]
            + enqueued_reqs_data[exp]["all"]["enqueued_reqs_forward_avg"]
        ) / enqueued_reqs_data[exp]["all"]["input_reqs_avg"]

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_avg"] = avg
    enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_std"] = std

    print(f"Algorithm {algorithm} done with {len(experiments)} experiments")


# GlobalNode case is special.
def upperbound_enqueued_request():
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms and environment.
        if "SingleDFaaS" in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print("WARN: Skipping GlobalNode, no experiments found!")
        return

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            enqueued_reqs_data[exp]["enqueued_reqs_local_avg"]
            / enqueued_reqs_data[exp]["input_reqs_avg"]
        )

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    enqueued_reqs_data_avg["GlobalNode"]["enqueued_reqs_ratios_avg"] = avg
    enqueued_reqs_data_avg["GlobalNode"]["enqueued_reqs_ratios_std"] = std

    print(f"GlobalNode done with {len(experiments)} experiments")


upperbound_enqueued_request()

### Average enqueued requests per episode

#### All agents

In [ ]:
def make_avg_enqueued_reqs_plot_all_agents():
    plt.close(fig="avg_enqueued_reqs_all_agents")
    fig = plt.figure(num="avg_enqueued_reqs_all_agents", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    iteration_coords_x = np.linspace(start=1, stop=300, num=300)

    for algorithm in algorithms + ["GlobalNode"]:
        if len(enqueued_reqs_data_avg[algorithm]) == 0:
            print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
            continue

        avg = enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_avg"]
        std = enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_std"]

        ax.plot(iteration_coords_x, avg, label=algorithm)
        ax.fill_between(iteration_coords_x, avg - std, avg + std, alpha=0.5)

    ax.set_title("Average enqueued requests per episode (all agents)")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Show 10% ticks.
    ax.set_ylim(0, 1)

    ax.set_xlabel("Iteration")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_avg_enqueued_reqs_plot_all_agents()

### Average enqueued requests per algorithm

In [ ]:
# Postprocess additional data.
enqueued_reqs_by_type_avg = {"SAC": {}, "PPO": {}, "APL": {}}
algorithms = ["SAC", "PPO", "APL"]
for algorithm in algorithms:
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms (and environment).
        if algorithm in exp_dir.name and "SingleDFaaS" not in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        continue

    ratios_local, ratios_forward, ratios_reject = [], [], []
    for exp in experiments:
        ratios_single_exp_local = (
            enqueued_reqs_data[exp]["all"]["enqueued_reqs_local_avg"]
            / enqueued_reqs_data[exp]["all"]["input_reqs_avg"]
        )
        ratios_single_exp_forward = (
            enqueued_reqs_data[exp]["all"]["enqueued_reqs_forward_avg"]
            / enqueued_reqs_data[exp]["all"]["input_reqs_avg"]
        )
        ratios_single_exp_reject = (
            enqueued_reqs_data[exp]["all"]["enqueued_reqs_reject_avg"]
            / enqueued_reqs_data[exp]["all"]["input_reqs_avg"]
        )

        ratios_local.append(ratios_single_exp_local)
        ratios_forward.append(ratios_single_exp_forward)
        ratios_reject.append(ratios_single_exp_reject)
    ratios_local = np.array(ratios_local)
    ratios_forward = np.array(ratios_forward)
    ratios_reject = np.array(ratios_reject)

    avg_local = np.average(ratios_local, axis=0)
    std_local = np.std(ratios_local, axis=0)

    avg_forward = np.average(ratios_forward, axis=0)
    std_forward = np.std(ratios_forward, axis=0)

    avg_reject = np.average(ratios_reject, axis=0)
    std_reject = np.std(ratios_reject, axis=0)

    enqueued_reqs_by_type_avg[algorithm]["Local"] = {"avg": avg_local, "std": std_local}
    enqueued_reqs_by_type_avg[algorithm]["Forwarded"] = {
        "avg": avg_forward,
        "std": std_forward,
    }
    enqueued_reqs_by_type_avg[algorithm]["Rejected"] = {
        "avg": avg_reject,
        "std": std_reject,
    }

    print(f"Algorithm {algorithm} done with {len(experiments)} experiments")

#### SAC

In [ ]:
def make_avg_enqueued_reqs_plot_by_type(algorithm):
    if len(enqueued_reqs_by_type_avg[algorithm]) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        return

    plt.close(fig=f"avg_enqueued_by_type_{algorithm}")
    fig = plt.figure(
        num=f"avg_enqueued_by_type_{algorithm}", layout="constrained", figsize=(7.68, 4)
    )
    fig.canvas.header_visible = False
    ax = fig.subplots()

    iteration_coords_x = np.linspace(start=1, stop=300, num=300)

    for reqs_type in ["Local", "Forwarded", "Rejected"]:
        avg = enqueued_reqs_by_type_avg[algorithm][reqs_type]["avg"]
        std = enqueued_reqs_by_type_avg[algorithm][reqs_type]["std"]

        ax.plot(iteration_coords_x, avg, label=reqs_type)
        ax.fill_between(iteration_coords_x, avg - std, avg + std, alpha=0.5)

        if reqs_type == "Rejected":
            avg_apl = enqueued_reqs_by_type_avg["APL"][reqs_type]["avg"]
            ax.plot(
                iteration_coords_x,
                avg_apl,
                label=f"{reqs_type} (APL)",
                color="green",
                linestyle="dotted",
                linewidth=3,
            )

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0, decimals=0))

    ax.set_xlabel("Iteration")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.
    data = enqueued_reqs_data  # Alias for better readability.

    # Save also the plot to disk as PDF.
    path = Path(f"plots/enqueued_requests_train_{algorithm}.pdf")
    fig.savefig(path)
    print(f"Figure saved to: {path.as_posix()!r}")


make_avg_enqueued_reqs_plot_by_type("SAC")

#### PPO

In [ ]:
make_avg_enqueued_reqs_plot_by_type("PPO")

## Processed requests

In [ ]:
# Common functions for processed requests.


def get_processed_requests_data_upperbound(iters):
    iterations = len(iters)
    data = {"input_reqs": np.empty(iterations), "processed_reqs": np.empty(iterations)}

    # Scan each iteration.
    for iter_idx in range(iterations):
        iter_data = iters[iter_idx]["env_runners"]

        episodes = iter_data["episodes_this_iter"]
        input_reqs_iter = np.empty(episodes)
        processed_local_iter = np.empty(episodes)

        # Iterate the episodes.
        for epi_idx in range(episodes):
            input_reqs = np.sum(
                iter_data["hist_stats"]["observation_input_requests"][epi_idx]
            )
            processed_local = np.sum(
                iter_data["hist_stats"]["processed_local"][epi_idx]
            )

            input_reqs_iter[epi_idx] = input_reqs
            processed_local_iter[epi_idx] = processed_local

        # Update iteration data (average the episodes values).
        data["input_reqs"][iter_idx] = np.average(input_reqs_iter)
        data["processed_reqs"][iter_idx] = np.average(processed_local_iter)

    return data


def get_processed_requests_data_episode(iter_data, epi_idx, env):
    episodes = iter_data["env_runners"]["episodes_this_iter"]
    iter_data = iter_data["env_runners"]["hist_stats"]

    data = defaultdict(lambda: defaultdict())
    (
        data["all"]["processed_reqs"],
        data["all"]["input_reqs"],
        data["all"]["processed_forwarded_reqs"],
    ) = (0, 0, 0)
    for agent in env.agents:
        processed_reqs = np.sum(iter_data["processed_local"][epi_idx][agent])
        input_reqs = np.sum(iter_data["observation_input_requests"][epi_idx][agent])

        try:
            processed_forward = np.sum(
                iter_data["processed_local_forward"][epi_idx][agent]
            )
        except (KeyError, IndexError):
            # May be missing if the agent did not receive any forwarded request.
            processed_forward = 0

        data[agent]["processed_reqs"] = processed_reqs
        data[agent]["input_reqs"] = input_reqs
        data[agent]["processed_forwarded_reqs"] = processed_forward
        data["all"]["processed_reqs"] += processed_reqs
        data["all"]["input_reqs"] += input_reqs
        data["all"]["processed_forwarded_reqs"] += processed_forward

    return data


def get_processed_requests_data(iter_data):
    #  is a disctionary with three levels of depth (experiment, iteration, metrics).
    final_data = defaultdict(lambda: defaultdict(lambda: defaultdict()))

    for exp_dir, iters in iter_data.items():
        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The GlobalNode data extraction is different from the normal flow.
            final_data[exp_dir] = get_processed_requests_data_upperbound(iters)
            continue

        agents = env.agents
        iterations = len(iters)

        # Create the portion of the dictionary for this experiment that
        # contains the average values of the metrics for each iteration.
        for agent in ["all"] + env.agents:
            for key in ["input_reqs", "processed_reqs", "processed_forwarded_reqs"]:
                final_data[exp_dir][agent][key] = np.empty(iterations)

        # For each iteration, calculate the metrics for each episode played,
        # then average the values for the number of episodes of that iteration.
        for iter_idx in range(iterations):
            episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]

            # Create the data dictionary that contains the metrics for each
            # episode in this iteration.
            data = defaultdict(lambda: defaultdict())
            for agent in ["all"] + env.agents:
                for key in ["input_reqs", "processed_reqs", "processed_forwarded_reqs"]:
                    data[agent][key] = np.empty(episodes, dtype=np.int32)

            # Iterate the episodes.
            for epi_idx in range(episodes):
                data_epi = get_processed_requests_data_episode(
                    iters[iter_idx], epi_idx, env
                )

                for agent in ["all"] + env.agents:
                    for key in [
                        "input_reqs",
                        "processed_reqs",
                        "processed_forwarded_reqs",
                    ]:
                        data[agent][key][epi_idx] = data_epi[agent][key]

            # Update iteration data.
            for agent in ["all"] + env.agents:
                for key in ["input_reqs", "processed_reqs", "processed_forwarded_reqs"]:
                    final_data[exp_dir][agent][key][iter_idx] = np.average(
                        data[agent][key]
                    )

    return final_data


processed_reqs_data = get_processed_requests_data(raw_exp_data)

In [ ]:
# Postprocess additional data.
processed_reqs_data_avg = {"SAC": {}, "PPO": {}, "APL": {}, "GlobalNode": {}}
algorithms = ["SAC", "PPO", "APL"]
for algorithm in algorithms:
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms (and environment).
        if algorithm in exp_dir.name and "SingleDFaaS" not in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        continue

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            processed_reqs_data[exp]["all"]["processed_reqs"]
            / processed_reqs_data[exp]["all"]["input_reqs"]
        )

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    processed_reqs_data_avg[algorithm]["processed_reqs_ratios_avg"] = avg
    processed_reqs_data_avg[algorithm]["processed_reqs_ratios_std"] = std

    print(f"Algorithm {algorithm} done with {len(experiments)} experiments")


# GlobalNode case is special.
def upperbound_processed_request():
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms and environment.
        if "SingleDFaaS" in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print("WARN: Skipping GlobalNode, no experiments found!")
        return

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            processed_reqs_data[exp]["processed_reqs"]
            / processed_reqs_data[exp]["input_reqs"]
        )

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    processed_reqs_data_avg["GlobalNode"]["processed_reqs_ratios_avg"] = avg
    processed_reqs_data_avg["GlobalNode"]["processed_reqs_ratios_std"] = std

    print(f"GlobalNode done with {len(experiments)} experiments")


upperbound_processed_request()

#### All agents

In [ ]:
def make_avg_processed_reqs_plot_all_agents():
    plt.close(fig="avg_processed_reqs_all_agents")
    fig = plt.figure(
        num="avg_processed_reqs_all_agents", layout="constrained", figsize=(7.68, 5)
    )
    fig.canvas.header_visible = False
    ax = fig.subplots()

    iteration_coords_x = np.linspace(start=1, stop=300, num=300)

    for algorithm in algorithms + ["GlobalNode"]:
        if len(processed_reqs_data_avg[algorithm]) == 0:
            print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
            continue

        avg = processed_reqs_data_avg[algorithm]["processed_reqs_ratios_avg"]
        std = processed_reqs_data_avg[algorithm]["processed_reqs_ratios_std"]

        ax.plot(iteration_coords_x, avg, label=algorithm)
        ax.fill_between(iteration_coords_x, avg - std, avg + std, alpha=0.5)

    # ax.set_title("Average processed requests per episode (all agents)")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Show 10% ticks.
    # figsize=(25, 6)ax.set_ylim(0, 1)

    ax.set_xlabel("Iteration")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.

    path = Path(f"plots/processed_requests_train.pdf")
    fig.savefig(path)
    print(f"Figure saved to: {path.as_posix()!r}")


make_avg_processed_reqs_plot_all_agents()

## Queue size

In [ ]:
# Common functions for queue.


def get_queue_data_upperbound(env, iters):
    iterations = len(iters)
    data = {
        "queue_size": np.empty(iterations),
        "queue_full": np.empty(iterations),
        "queue_size_std": np.empty(iterations),
    }

    # Scan each iteration.
    for iter_idx in range(iterations):
        iter_data = iters[iter_idx]["env_runners"]

        episodes = iter_data["episodes_this_iter"]

        queue_size = np.empty(episodes)
        queue_size_std = np.empty(episodes)
        queue_full = np.empty(episodes)

        # Iterate the episodes.
        for epi_idx in range(episodes):
            queue = np.array(iter_data["hist_stats"]["queue_size"][epi_idx])

            queue_size[epi_idx] = np.mean(queue)
            queue_size_std[epi_idx] = np.std(queue)
            queue_full[epi_idx] = np.where(queue == env.queue_capacity)[0].size

        # Update iteration data (average the episodes values).
        data["queue_size"][iter_idx] = np.average(queue_size)
        data["queue_size_std"][iter_idx] = np.average(queue_size_std)
        data["queue_full"][iter_idx] = np.average(queue_full)

    return data


def get_queue_data(iter_data):
    #  is a disctionary with three levels of depth (experiment, iteration, metrics).
    final_data = defaultdict(lambda: defaultdict(lambda: defaultdict()))

    for exp_dir, iters in iter_data.items():
        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The GlobalNode data extraction is different from the normal flow.
            final_data[exp_dir] = get_queue_data_upperbound(env, iters)
            continue

        iterations = len(iters)

        # Keys of the dictionary for each agent.
        metrics = ["queue_size_mean", "queue_size_std", "queue_full"]

        # Create the portion of the dictionary for this experiment that
        # contains the average values of the metrics for each iteration.
        for agent in ["all"] + env.agents:
            for metric in metrics:
                final_data[exp_dir][agent][metric] = np.empty(iterations)

        # For each iteration, calculate the metrics for each episode played,
        # then average the values for the number of episodes of that iteration.
        for iter_idx in range(iterations):
            episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]
            hist_stats = iters[iter_idx]["env_runners"]["hist_stats"]

            # Temporary dictionary for each iteration.
            epi_data = defaultdict(lambda: defaultdict())
            for agent in ["all"] + env.agents:
                for metric in metrics:
                    epi_data[agent][metric] = np.empty(episodes)

            # Iterate the episodes.
            for epi_idx in range(episodes):
                queue_all = np.empty(0)

                for agent in env.agents:
                    queue = np.array(hist_stats["queue_size"][epi_idx][agent])
                    queue_all = np.concatenate([queue_all, queue])

                    epi_data[agent]["queue_size_mean"][epi_idx] = np.mean(queue)
                    epi_data[agent]["queue_size_std"][epi_idx] = np.std(queue)

                    # Count the number of occurencies where queue is full
                    epi_data[agent]["queue_full"][epi_idx] = np.where(
                        queue == env.queue_capacity
                    )[0].size

                epi_data["all"]["queue_size_mean"][epi_idx] = np.mean(queue_all)
                epi_data["all"]["queue_size_std"][epi_idx] = np.std(queue_all)
                epi_data["all"]["queue_full"][epi_idx] = np.where(
                    queue == env.queue_capacity
                )[0].size

            # Update iteration data.
            for agent in ["all"] + env.agents:
                for metric in metrics:
                    mean = np.mean(epi_data[agent][metric])
                    final_data[exp_dir][agent][metric][iter_idx] = mean

    return final_data


queue_data = get_queue_data(raw_exp_data)

### Average queue size per episode

In [ ]:
# Postprocess additional data.
queue_size_avg = {"SAC": {}, "PPO": {}, "APL": {}}
algorithms = ["SAC", "PPO", "APL"]
for algorithm in algorithms:
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms (and environment).
        if algorithm in exp_dir.name and "SingleDFaaS" not in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        continue

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            queue_data[exp]["all"]["queue_size_mean"] / env.queue_capacity
        )

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    queue_size_avg[algorithm]["queue_size_ratios_avg"] = avg
    queue_size_avg[algorithm]["queue_size_ratios_std"] = std

    print(f"Algorithm {algorithm} done with {len(experiments)} experiments")

#### All agents

In [ ]:
def make_avg_queue_size_plot_all_agents():
    plt.close(fig="avg_queue_size_all_agents")
    fig = plt.figure(
        num="avg_queue_size_all_agents", layout="constrained", figsize=(7.68, 4)
    )
    fig.canvas.header_visible = False
    ax = fig.subplots()

    iteration_coords_x = np.linspace(start=1, stop=300, num=300)

    for algorithm in algorithms:
        if len(enqueued_reqs_data_avg[algorithm]) == 0:
            print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
            continue

        avg = queue_size_avg[algorithm]["queue_size_ratios_avg"]
        std = queue_size_avg[algorithm]["queue_size_ratios_std"]

        ax.plot(iteration_coords_x, avg, label=algorithm)
        ax.fill_between(iteration_coords_x, avg - std, avg + std, alpha=0.5)

    # ax.set_title("Average queue size per episode (all agents)")

    ax.set_ylabel("Requests in queue")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Show 10% ticks.
    # ax.set_ylim(0, 0.5)

    ax.set_xlabel("Iteration")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.

    path = Path(f"plots/queue_size_train.pdf")
    fig.savefig(path)
    print(f"Figure saved to: {path.as_posix()!r}")


make_avg_queue_size_plot_all_agents()

# Evaluation

## Experiment loading

In [ ]:
prefix_dir = Path("../results/paise2025_2")

exps_dir = []
for exp in prefix_dir.iterdir():
    exps_dir.append(exp)

# DEBUG
# exps_dir = [prefix_dir / "DFAAS-MA_2025-02-13_02-11-18_PPO_45"]

assert len(exps_dir) > 0, "must select at least one experiment"

# Preload the data (evaluation.json file) for all selected experiments.
raw_eval_data = {}
for exp_dir in exps_dir:
    raw_eval_data[exp_dir] = dfaas_utils.parse_result_file(exp_dir / "evaluation.json")

# Create the reference environment based on DFaaS.
for exp_dir in exps_dir:
    env = base.get_env(exp_dir)
    if env.__class__ == dfaas_env.DFaaS:
        break

# At least one experiment must be of type DFaaS. SingleDFaaS is used only as
# reference.
assert env.__class__ == dfaas_env.DFaaS, f"{env.__class__}"

print("Selected experiments:")
for exp_dir in exps_dir:
    print(f"  - {exp_dir.name}")

## Enqueued requests

In [ ]:
# Common functions for processed requests.


def get_enqueued_requests_data_globalnode_eval(evals):
    episodes = evals[0]["env_runners"]["num_episodes"]
    eval_data = evals[0]["env_runners"]["hist_stats"]
    assert episodes > 0

    data = {}
    data["input_reqs"] = np.zeros(episodes)
    data["enqueued_reqs_local"] = np.zeros(episodes)
    data["enqueued_reqs_reject"] = np.zeros(episodes)

    # Iterate the episodes.
    for epi_idx in range(episodes):
        input_reqs = np.sum(eval_data["observation_input_requests"][epi_idx])

        # Calculate local, queued, or directly processed requests. Note that the
        # rejections must be subtracted!
        local_reqs = np.array(eval_data["action_local"][epi_idx])
        local_rejects = np.array(eval_data["local_rejects_queue_full"][epi_idx])
        real_local_reqs = np.sum(local_reqs - local_rejects)

        real_rejected_reqs = np.sum(local_rejects)

        assert np.all(real_local_reqs + real_rejected_reqs == input_reqs)

        data["input_reqs"][epi_idx] = input_reqs
        data["enqueued_reqs_local"][epi_idx] = real_local_reqs
        data["enqueued_reqs_reject"] = real_rejected_reqs

    # Update final data.
    final_data = {}
    final_data["input_reqs_avg"] = np.average(data["input_reqs"])
    final_data["enqueued_reqs_local_avg"] = np.average(data["enqueued_reqs_local"])
    final_data["enqueued_reqs_forward_avg"] = 0
    final_data["enqueued_reqs_reject_avg"] = np.average(data["enqueued_reqs_reject"])

    return final_data


def get_enqueued_requests_data_eval(raw_eval_data):
    # (experiment, metrics).
    final_data = defaultdict(lambda: defaultdict())

    for exp_dir, evals in raw_eval_data.items():
        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The GlobalNode data extraction is different from the normal flow.
            final_data[exp_dir] = get_enqueued_requests_data_globalnode_eval(evals)
            continue

        episodes = evals[0]["env_runners"]["num_episodes"]
        eval_data = evals[0]["env_runners"]["hist_stats"]
        assert episodes > 0

        data = {}
        data["input_reqs"] = np.zeros(episodes)
        data["enqueued_reqs_local"] = np.zeros(episodes)
        data["enqueued_reqs_forward"] = np.zeros(episodes)
        data["enqueued_reqs_reject"] = np.zeros(episodes)

        # Iterate the episodes.
        for epi_idx in range(episodes):
            for agent in env.agents:
                input_reqs = np.sum(
                    eval_data["observation_input_requests"][epi_idx][agent]
                )

                # Calculate local, queued, or directly processed requests. Note that the
                # rejections must be subtracted!
                local_reqs = np.array(eval_data["action_local"][epi_idx][agent])
                local_rejects = np.array(
                    eval_data["local_rejects_queue_full"][epi_idx][agent]
                )
                real_local_reqs = np.sum(local_reqs - local_rejects)

                # Same for forwarded requests.
                forwarded_reqs = np.array(eval_data["action_forward"][epi_idx][agent])
                forwarded_rejects = np.array(
                    eval_data["forward_rejects"][epi_idx][agent]
                )
                real_forwarded_reqs = np.sum(forwarded_reqs - forwarded_rejects)

                rejected_reqs = np.array(eval_data["action_reject"][epi_idx][agent])
                real_rejected_reqs = np.sum(
                    rejected_reqs + local_rejects + forwarded_rejects
                )

                assert np.all(
                    real_local_reqs + real_forwarded_reqs + real_rejected_reqs
                    == input_reqs
                )

                data["input_reqs"][epi_idx] += input_reqs
                data["enqueued_reqs_local"][epi_idx] += real_local_reqs
                data["enqueued_reqs_forward"][epi_idx] += real_forwarded_reqs
                data["enqueued_reqs_reject"][epi_idx] += real_rejected_reqs

        # Update final data.
        final_data[exp_dir]["input_reqs_avg"] = np.average(data["input_reqs"])
        final_data[exp_dir]["enqueued_reqs_local_avg"] = np.average(
            data["enqueued_reqs_local"]
        )
        final_data[exp_dir]["enqueued_reqs_forward_avg"] = np.average(
            data["enqueued_reqs_forward"]
        )
        final_data[exp_dir]["enqueued_reqs_reject_avg"] = np.average(
            data["enqueued_reqs_reject"]
        )

    return final_data


enqueued_reqs_data = get_enqueued_requests_data_eval(raw_eval_data)

In [ ]:
# Postprocess additional data.
enqueued_reqs_by_type_avg = {"SAC": {}, "PPO": {}, "APL": {}, "GlobalNode": {}}
algorithms = ["SAC", "PPO", "APL", "GlobalNode"]
for algorithm in algorithms:
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms (and environment).
        if algorithm == "GlobalNode" and "SingleDFaaS" in exp_dir.name:
            experiments.append(exp_dir)
            continue
        if algorithm in exp_dir.name and "SingleDFaaS" not in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        continue

    ratios_local, ratios_forward, ratios_reject = [], [], []
    for exp in experiments:
        ratios_single_exp_local = (
            enqueued_reqs_data[exp]["enqueued_reqs_local_avg"]
            / enqueued_reqs_data[exp]["input_reqs_avg"]
        )
        ratios_single_exp_forward = (
            enqueued_reqs_data[exp]["enqueued_reqs_forward_avg"]
            / enqueued_reqs_data[exp]["input_reqs_avg"]
        )
        ratios_single_exp_reject = (
            enqueued_reqs_data[exp]["enqueued_reqs_reject_avg"]
            / enqueued_reqs_data[exp]["input_reqs_avg"]
        )

        ratios_local.append(ratios_single_exp_local)
        ratios_forward.append(ratios_single_exp_forward)
        ratios_reject.append(ratios_single_exp_reject)
    ratios_local = np.array(ratios_local)
    ratios_forward = np.array(ratios_forward)
    ratios_reject = np.array(ratios_reject)

    avg_local = np.average(ratios_local, axis=0)
    std_local = np.std(ratios_local, axis=0)

    avg_forward = np.average(ratios_forward, axis=0)
    std_forward = np.std(ratios_forward, axis=0)

    avg_reject = np.average(ratios_reject, axis=0)
    std_reject = np.std(ratios_reject, axis=0)

    enqueued_reqs_by_type_avg[algorithm]["Local"] = {"avg": avg_local, "std": std_local}
    enqueued_reqs_by_type_avg[algorithm]["Forwarded"] = {
        "avg": avg_forward,
        "std": std_forward,
    }
    enqueued_reqs_by_type_avg[algorithm]["Rejected"] = {
        "avg": avg_reject,
        "std": std_reject,
    }

    print(f"Algorithm {algorithm} done with {len(experiments)} experiments")

In [ ]:
for algorithm in enqueued_reqs_by_type_avg.keys():
    print("Algorithm:", algorithm)
    for reqs_type in enqueued_reqs_by_type_avg[algorithm].keys():
        print(
            f"    {reqs_type}: {enqueued_reqs_by_type_avg[algorithm][reqs_type]['avg']*100}"
        )